# Generierung der Bilder

Zum generieren der Bilder wurde StableDiffusionXL verwendet. Dies ist ein bereits vorgefertigtes Modell zur generierung von Bilder. Dieses Modell ist zur Zeit eines der meist-verwendetsten Modelle.


In [ ]:
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from diffusers import DiffusionPipeline

In [1]:
# Initialisiere CLIP und Diffusion-Modell
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
diffusion_model = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16").to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
source_dir = './images/car/real/'
target_dir = './images/car/fake_ImgToImg/'

In [ ]:
# Erstelle den Zielordner, falls er nicht existiert
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

In [ ]:
def resize_and_crop(image, target_size=(256, 256)):
    width, height = img.size
    # kleineren Wert ermitteln, um das Bild nicht zu verzerren aber auch gleichzeitig so wenig wie möglich an Info zu verlieren 
    new_size = min(width, height)
    
    left = (width - new_size) / 2
    top = (height - new_size) / 2
    right = (width + new_size) / 2
    bottom = (height + new_size) / 2
    
    # Bild croppen und resizen 
    img_cropped = img.crop((left, top, right, bottom))
    img_resized = img_cropped.resize(target_size, Image.LANCZOS)
    
    return img_resized

In [ ]:
# Definiere Prompts für die Bildbeschreibung
prompts = [
    "new car",
    "old car",
]

# Durchlaufe alle Dateien im Quellordner
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        init_image = Image.open(file_path)
        init_image = resize_and_crop(init_image)

        # Verwende CLIP, um einen beschreibenden Prompt für das Bild zu generieren
        inputs = clip_processor(text=prompts, images=init_image, return_tensors="pt")
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        best_prompt_index = probs.argmax()
        prompt_with_label = prompts[best_prompt_index]

        # Verwende den beschreibenden Prompt für die Bildgenerierung mit dem Diffusion-Modell
        generated_image = diffusion_model(prompt_with_label, image=init_image, strength=0.5, guidance_scale=8).images[0]

        # Speichere das generierte Bild
        save_path = os.path.join(target_dir, 'generated_' + prompt_with_label.replace(" ", "_") + filename)
        generated_image.save(save_path)

        print(f"Bild generiert und gespeichert als: {save_path}")